In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Set the working directory
cbis_path = '/content/drive/MyDrive/colab_scripts/repository/datasets'
os.chdir(cbis_path)

# Confirm current directory
print("Current Directory:", os.getcwd())

Current Directory: /content/drive/MyDrive/colab_scripts/repository/datasets


In [ ]:
import os
import cv2
import pandas as pd
from tqdm import tqdm  # Optional: progress bar

In [ ]:
# === CONFIGURATION ===
CSV_PATH = 'CBIS-DDSM/csv/mass_train_jpg2.csv'        # Path to your CSV
MASK_COL = 'jpg ROI mask file path'                   # Column containing ROI mask paths
OUTPUT_CSV = 'CBIS-DDSM/csv/mass_train_jpg2_bbox.csv'


# === FUNCTION: Extract YOLO bbox from a binary mask ===
def get_yolo_bbox(mask):
    _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None

    x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    h_img, w_img = mask.shape
    return (
        round((x + w / 2) / w_img, 6),
        round((y + h / 2) / h_img, 6),
        round(w / w_img, 6),
        round(h / h_img, 6)
    )

# === MAIN PROCESS ===
df = pd.read_csv(CSV_PATH)
yolo_bboxes = []

print("🔍 Processing masks and extracting YOLO bounding boxes...")

for _, row in tqdm(df.iterrows(), total=len(df)):
    mask_path = row[MASK_COL]
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if mask is None:
        print(f"❌ Could not read image: {mask_path}")
        yolo_bboxes.append(None)
        continue

    bbox = get_yolo_bbox(mask)
    if bbox is None:
        print(f"⚠️ No contour found in: {mask_path}")

    yolo_bboxes.append(bbox)

# Add new column with YOLO bbox tuples
df['yolo_bbox'] = yolo_bboxes

# Save updated DataFrame
df.to_csv(OUTPUT_CSV, index=False)
print(f"\n✅ Updated CSV with YOLO bounding boxes saved to: {OUTPUT_CSV}")


🔍 Processing masks and extracting YOLO bounding boxes...


100%|██████████| 1253/1253 [09:36<00:00,  2.17it/s]


✅ Updated CSV with YOLO bounding boxes saved to: CBIS-DDSM/csv/mass_train_jpg2_bbox.csv
